In [2]:
import pandas as pd

file = '/Users/pawanjeetkaur/Downloads/SPRING 2020/Big Data - IDS 561/HW_4/u.data'

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

import collections

sc = SparkContext("local", "First App")
sql = SQLContext(sc)
spark = SparkSession.builder.master("local").appName("movie_lens").getOrCreate()

In [3]:
lines = sc.textFile(file)
type(lines)
lines.take(4)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923']

In [4]:
clean_data = lines.map(lambda x:x.split('\t'))

In [5]:
clean_data.take(4)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923']]

In [6]:
# extract ratings 
rate = clean_data.map(lambda y: int(y[2]))
rate.mean()

3.529859999999947

In [7]:
#extract users
users = clean_data.map(lambda y: int(y[0]))
users.distinct().count()

943

In [8]:
# count of distinct movies
clean_data.map(lambda y: int(y[1])).distinct().count() 

1682

In [9]:
# import the models to be used
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel, Rating

In [10]:
mls = lines.map(lambda l: l.split('\t'))

ratings = mls.map(lambda x: Rating(int(x[0]),int(x[1]), float(x[2])))

In [11]:
import numpy
train, test = ratings.randomSplit([0.7,0.3],7856)

print(train.count())
print(test.count())

69958
30042


In [12]:
# cache train/test data
train.cache()
test.cache()

PythonRDD[18] at RDD at PythonRDD.scala:53

In [13]:
# set up paramters for ALS 
rank = 5
numIterations = 10 

#Create the model on the training data
model = ALS.train(train, rank, numIterations)

In [14]:
#Examine the latent features for one product
model.productFeatures().first()

(1,
 array('d', [1.0812033414840698, -0.6098031997680664, 1.3819518089294434, -0.8973950147628784, -0.4324202537536621]))

In [15]:
#Examine the latent features for one user
model.userFeatures().first()

(1,
 array('d', [0.9891213178634644, -1.1177046298980713, 1.2366325855255127, -0.07294294238090515, -0.4959731996059418]))

In [16]:
# For Product X, Find N Users to Sell To
model.recommendUsers(242,100)

[Rating(user=4, product=242, rating=5.631577865237972),
 Rating(user=240, product=242, rating=5.581940090691871),
 Rating(user=928, product=242, rating=5.481553096563971),
 Rating(user=310, product=242, rating=5.467094812980218),
 Rating(user=803, product=242, rating=5.3541405630863075),
 Rating(user=414, product=242, rating=5.221634918875797),
 Rating(user=534, product=242, rating=5.181642097359205),
 Rating(user=180, product=242, rating=5.176824877557046),
 Rating(user=45, product=242, rating=5.162188852505334),
 Rating(user=219, product=242, rating=5.138261770307382),
 Rating(user=386, product=242, rating=5.103824995357051),
 Rating(user=118, product=242, rating=5.0584736273542354),
 Rating(user=353, product=242, rating=5.057665883542374),
 Rating(user=241, product=242, rating=5.050767024610874),
 Rating(user=357, product=242, rating=4.967344322327577),
 Rating(user=261, product=242, rating=4.964578758465336),
 Rating(user=167, product=242, rating=4.958858678970751),
 Rating(user=80

In [17]:
# For User Y Find N Products to Promote
model.recommendProducts(196,10)

[Rating(user=196, product=1438, rating=9.545188255293116),
 Rating(user=196, product=1160, rating=8.947953921077355),
 Rating(user=196, product=1315, rating=8.443920506147968),
 Rating(user=196, product=1298, rating=8.12891011225617),
 Rating(user=196, product=1176, rating=8.08221832370801),
 Rating(user=196, product=1426, rating=7.869675065244834),
 Rating(user=196, product=1085, rating=7.788261125906321),
 Rating(user=196, product=1184, rating=7.733133794006033),
 Rating(user=196, product=1335, rating=7.615182806028126),
 Rating(user=196, product=1427, rating=7.421774503650177)]

In [18]:
#Predict Single Product for Single User
model.predict(196, 242)

4.248268946199023

In [19]:
# Predict Multi Users and Multi Products
# Pre-Processing
pred_input = train.map(lambda x:(x[0],x[1]))   

In [20]:
# Lots of Predictions
#Returns Ratings(user, item, prediction)
pred = model.predictAll(pred_input) 

In [21]:
#Get Performance Estimate
#Organize the data to make (user, product) the key)

true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

In [22]:
true_pred = true_reorg.join(pred_reorg)

In [23]:
from math import sqrt

MSE = true_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)

In [24]:
test_input = test.map(lambda x:(x[0],x[1])) 
pred_test = model.predictAll(test_input)

test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))

test_pred = test_reorg.join(pred_reorg)

test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)

In [25]:
print(test_MSE)
print(test_RMSE)

1.023861138871279
1.011860236826845


In [26]:
seed = 5
iterations = 10
#regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(train, rank, seed = seed, iterations = numIterations)
    
    test_input = test.map(lambda x:(x[0],x[1])) 
    pred_test = model.predictAll(test_input)

    test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
    pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))

    rates_and_preds = test_reorg.join(pred_reorg)

    #rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    
    print("For rank", rank, "the RMSE is", error)
    if error < min_error:
        min_error = error
        best_rank = rank

print("The best model was trained with rank", best_rank)

For rank 4 the RMSE is 0.9915248794772764
For rank 8 the RMSE is 1.0759270988028544
For rank 12 the RMSE is 1.151978488010925
The best model was trained with rank 4


In [43]:
#best_model = ALS.train(train, best_rank, seed = seed, iterations = numIterations, coldStartStrategy = "drop")

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

train_df = train.toDF()
test_df = test.toDF()

als = ALS(maxIter=numIterations,rank=best_rank,userCol="user", itemCol="product", 
                 ratingCol="rating",coldStartStrategy="drop")
best_model = als.fit(train_df)

In [44]:
predictions_test_best = best_model.transform(test_df)
evaluator_test_best = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

rmse = evaluator_test_best.evaluate(predictions_test_best)

print(rmse)

0.9266849479886077


In [46]:
rmse

0.9266849479886077

In [47]:
userRecs = best_model.recommendForAllUsers(10)
print(userRecs.count())
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)
print(movieRecs.count())

943
1631


In [48]:
userRecs_df = userRecs.toPandas()
print(userRecs_df.shape)

movieRecs_df = movieRecs.toPandas()
print(movieRecs_df.shape)

(943, 2)
(1631, 2)


In [49]:
import string

movieid=list()
for i in range(0,943):
  l1=list()
  for j in range(0,10):
    l1.append(userRecs_df['recommendations'][i][j][0])
  
  s = [str(i) for i in l1]
  res = (",".join(s))
  movieid.append(res)

userRecs_df['final_recommend']=movieid

In [50]:
userRecs_df
userRecs_df.drop(["recommendations"], axis = 1, inplace = True) 

In [51]:
#Exporting the output

#import os
#os.chdir('/content/drive/My Drive/Big_data_project/')

userRecs_df.to_csv('Assignment_3_op.csv',sep='\t',index=False,header=False)

In [52]:
userRecs_df

,user,final_recommend
0,471,"1192,394,1391,1038,1394,958,1167,1242,454,681"
1,463,"1167,1599,1398,1099,1084,1192,1062,1585,1553,958"
2,833,"1463,1643,1462,1367,1536,853,1405,1558,641,1499"
3,496,"913,75,1500,1405,1367,1143,793,1410,1368,899"
4,148,"1500,1463,1367,1643,793,1449,75,913,1143,1410"
...,...,...
938,208,"1472,1063,1463,814,867,1099,1166,1242,496,1167"
939,401,"1167,1242,1166,867,1099,814,1160,1063,1472,496"
940,422,"1463,1367,1449,1643,1500,1651,1636,1631,814,1122"
941,517,"1631,1651,1636,1367,1449,1293,1398,1500,1169,169"
